In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras.callbacks import Callback

import pandas as pd
import cirq
import sympy
import numpy as np
import scipy
from scipy.linalg import eigh, cholesky
from scipy.stats import norm, multivariate_normal

from pylab import plot, show, axis, subplot, xlabel, ylabel, grid

# visualization tools
import seaborn as sns
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit
import IPython

sns.set()
plt.rcParams['figure.figsize'] = [10, 5]
InteractiveShell = IPython.core.interactiveshell.InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
tf.random.set_seed(5)

In [66]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


class VThresh(keras.layers.Layer):
    """Parameterized quantum signal processing layer.

    The `QSP` layer implements the quantum signal processing circuit with trainable QSP angles.

    The input of the layer is/are theta(s) where x = cos(theta), and w(x) is X rotation in the QSP sequence.

    The output is the real part of the upper left element in the resulting unitary that describes the whole sequence.
    This is Re[P(x)] in the representation of the QSP unitary from Gilyen et al.

    Input is of the form:
    [[theta1], [theta2], ... ]

    Output is of the form:
    [[P(x1)], [P(x2)], ...]

    The layer requires the desired polynomial degree of P(x)

    """

    def __init__(self, poly_deg=0):
        """
        Params
        ------
        poly_deg: The desired degree of the polynomial in the QSP sequence.
            the layer will be parameterized with poly_deg + 1 trainable phi.
        """
        super(VThresh, self).__init__()
        self.poly_deg = poly_deg
        phi_init = tf.random_uniform_initializer(minval=0, maxval=np.pi)
        self.phis = tf.Variable(
            initial_value=phi_init(shape=(poly_deg, 1), dtype=tf.float32),
            trainable=True,
        )
    
    def unitary(self, inp):
        """
        Shadow QSP 
        q0 ---
        q1 ---
        Order 
            phi[q1], controlled U[q0,q1], V[q0]
            d times
        """
        th = inp[:,None,0]
        eps = inp[:,None,1]
        batch_dim = tf.gather(tf.shape(th), 0)
        
        
        # tiled up X rotations (input W(x))
        px = tf.constant([[0.0, 1], [1, 0]], dtype=tf.complex64)
        px = tf.expand_dims(px, axis=0)
        px = tf.repeat(px, [batch_dim], axis=0)
        
        rot_x_arg = tf.complex(real=0.0, imag=th)
        rot_x_arg = tf.expand_dims(rot_x_arg, axis=1)
        rot_x_arg = tf.tile(rot_x_arg, [1, 2, 2])
        

        wx = tf.linalg.LinearOperatorFullMatrix(tf.linalg.expm(tf.multiply(px, rot_x_arg)))

        c_op = tf.linalg.LinearOperatorFullMatrix(tf.constant([[0, 0], [0, 1]], dtype=tf.complex64))
        c_op = tf.linalg.LinearOperatorKronecker([c_op, wx])
        i_op = tf.constant([[[1.0,0,0,0], 
                            [0,1,0,0],
                            [0,0,0,0],
                            [0,0,0,0]]], dtype=tf.complex64)
        i_op = tf.repeat(i_op, [batch_dim], axis=0)
        c_op = c_op.to_dense() + i_op
        
        # tiled up V rotations

        px = tf.constant([[0.0, 1], [1, 0]], dtype=tf.complex64)
        px = tf.expand_dims(px, axis=0)
        px = tf.repeat(px, [batch_dim], axis=0)
        
        
        rot_v_arg = tf.complex(real=0.0, imag=eps)
        rot_v_arg = tf.expand_dims(rot_v_arg, axis=1)
        rot_v_arg = tf.tile(rot_v_arg, [1, 2, 2])
        
        rv = tf.linalg.LinearOperatorFullMatrix(tf.linalg.expm(tf.multiply(px, rot_v_arg)))

        pi = tf.linalg.LinearOperatorFullMatrix(tf.constant([[1, 0], [0, 1]], dtype=tf.complex64))
        v_op = tf.linalg.LinearOperatorKronecker([rv, pi]).to_dense()
        
        
        # tiled up Z rotations
        pz = tf.constant([[1.0, 0], [0, -1]], dtype=tf.complex64)
        pz = tf.expand_dims(pz, axis=0)
        pz = tf.repeat(pz, [batch_dim], axis=0)

        z_rotations = []
        for k in range(self.poly_deg + 1):
            phi = self.phis[k]
            rot_z_arg = tf.complex(real=0.0, imag=phi)
            rot_z_arg = tf.expand_dims(rot_z_arg, axis=0)
            rot_z_arg = tf.expand_dims(rot_z_arg, axis=0)
            rot_z_arg = tf.tile(rot_z_arg, [batch_dim, 2, 2])

            rz = tf.linalg.LinearOperatorFullMatrix(tf.linalg.expm(tf.multiply(pz, rot_z_arg)))
            pi = tf.linalg.LinearOperatorFullMatrix(tf.constant([[1, 0], [0, 1]], dtype=tf.complex64))
            rz = tf.linalg.LinearOperatorKronecker([pi, rz]).to_dense()
            z_rotations.append(rz)
            
        u = z_rotations[0]
        u = tf.matmul(c_op, u)
        u = tf.matmul(v_op, u)
        for rz in z_rotations[1:]:
            u = tf.matmul(rz, u)
            u = tf.matmul(c_op, u)
            u = tf.matmul(v_op, u)
        
        
        # initial state 
        init_state = self.random_init_state(batch_dim)
        
        final_state = tf.matmul(u,init_state)
        return tf.norm(final_state[:,:2],axis=1), tf.norm(final_state[:,2:],axis=1)
        
    
    def random_init_state(self, batch_dim):
        px = tf.tile(tf.constant([[[0.0,1], [1,0]]], dtype=tf.complex64),[3,1,1])

        init_env_rot_ang = tf.tile(
            tf.complex(
                real=0.0, 
                imag=tf.random.uniform(
                    (3,1,1), minval=0, maxval=np.pi/4, dtype=tf.dtypes.float32
                )
            ),
            [1,2,2]
        )

        init_env_rot_ang
        init_env_rot = tf.linalg.LinearOperatorFullMatrix(tf.linalg.expm(tf.multiply(px, init_env_rot_ang)))
        pi = tf.linalg.LinearOperatorFullMatrix(tf.constant([[1,0],[0,1]], dtype=tf.complex64))
        init_env_rot = tf.linalg.LinearOperatorKronecker([init_env_rot,pi]).to_dense()
        init_state = tf.matmul(init_env_rot, [[1],[0],[0],[0]])
        
        return init_state
    
    def call(self, inp):
        return tf.abs(self.unitary(inp))
#         e = tf.matvec(u,s)
#         e = tf.matvec(f,u)
#         return tf.norm(e)

def frobenius_norm(y,x):
    u = x-y
    u = tf.norm(u, ord='euclidean',axis=1)
    u = tf.norm(u, ord='euclidean',axis=1)
    return u


class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            print("Early stopping requires %s available!" % self.monitor)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True
            
            
def construct_qsp_model2(poly_deg):
    """Helper function that compiles a QSP model with mean squared error and adam optimizer.

    Params
    ------
    poly_deg : int
        the desired degree of the polynomial in the QSP sequence.

    Returns
    -------
    Keras model
        a compiled keras model with trainable phis in a poly_deg QSP sequence.
    """
    theta_input = tf.keras.Input(shape=(2,), dtype=tf.float32, name="theta")
    qsp = VThresh(poly_deg)
    u = qsp(theta_input)
    model = tf.keras.Model(inputs=theta_input, outputs=u)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
    loss = tf.keras.losses.MeanSquaredError()
    model.compile(optimizer=optimizer, loss=loss)
    return qsp, model


In [59]:
px = tf.tile(tf.constant([[[0.0,1], [1,0]]], dtype=tf.complex64),[3,1,1])

init_env_rot_ang = tf.tile(
    tf.complex(
        real=0.0, 
        imag=tf.random.uniform(
            (3,1,1), minval=0, maxval=np.pi/4, dtype=tf.dtypes.float32
        )
    ),
    [1,2,2]
)

init_env_rot_ang
init_env_rot = tf.linalg.LinearOperatorFullMatrix(tf.linalg.expm(tf.multiply(px, init_env_rot_ang)))
pi = tf.linalg.LinearOperatorFullMatrix(tf.constant([[1,0],[0,1]], dtype=tf.complex64))
init_env_rot = tf.linalg.LinearOperatorKronecker([init_env_rot,pi]).to_dense()
init_env_state = tf.matmul(init_env_rot, [[1],[0],[0],[0]])
init_env_state
init_env_state[:,0:2]

tf.norm(init_env_state[:,2:],axis=1)

<tf.Tensor: shape=(3, 2, 2), dtype=complex64, numpy=
array([[[0.+0.24222863j, 0.+0.24222863j],
        [0.+0.24222863j, 0.+0.24222863j]],

       [[0.+0.18592186j, 0.+0.18592186j],
        [0.+0.18592186j, 0.+0.18592186j]],

       [[0.+0.2805272j , 0.+0.2805272j ],
        [0.+0.2805272j , 0.+0.2805272j ]]], dtype=complex64)>

<tf.Tensor: shape=(3, 4, 1), dtype=complex64, numpy=
array([[[0.97080576+0.j        ],
        [0.        +0.j        ],
        [0.        +0.23986678j],
        [0.        +0.j        ]],

       [[0.98276633+0.j        ],
        [0.        +0.j        ],
        [0.        +0.18485259j],
        [0.        +0.j        ]],

       [[0.96090966+0.j        ],
        [0.        +0.j        ],
        [0.        +0.27686226j],
        [0.        +0.j        ]]], dtype=complex64)>

<tf.Tensor: shape=(3, 2, 1), dtype=complex64, numpy=
array([[[0.97080576+0.j],
        [0.        +0.j]],

       [[0.98276633+0.j],
        [0.        +0.j]],

       [[0.96090966+0.j],
        [0.        +0.j]]], dtype=complex64)>

<tf.Tensor: shape=(3, 1), dtype=complex64, numpy=
array([[0.23986678+0.j],
       [0.18485259+0.j],
       [0.27686226+0.j]], dtype=complex64)>

In [67]:
n = 3
theta = np.pi/n
nv = 100
v = np.pi/nv # x rotation
poly_deg = 12

# The intput theta training values 
th_in = tf.constant([[theta,np.pi/4],[theta,0]],dtype=tf.float32)
# The desired real part of p(x) which is the upper left value in the unitary of the qsp sequence
expected_outputs = tf.constant([[1,0],[0,1]], dtype=tf.float32)


qsp, model = construct_qsp_model2(poly_deg)
history = model.fit(x=th_in, y=expected_outputs, epochs=500)


loss = pd.Series(history.history['loss'])
plt.plot(loss.rolling(40).mean().dropna())
print(qsp.unitary(th_in))

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


ValueError: in user code:

    <ipython-input-66-58ed8127dbab>:149 call  *
        return tf.abs(self.unitary(inp))
    <ipython-input-66-58ed8127dbab>:100 unitary  *
        phi = self.phis[k]
    /Users/jdocter/opt/anaconda3/envs/qsp/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1293 _SliceHelperVar
        return _slice_helper(var.value(), slice_spec, var)
    /Users/jdocter/opt/anaconda3/envs/qsp/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /Users/jdocter/opt/anaconda3/envs/qsp/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1036 _slice_helper
        return strided_slice(
    /Users/jdocter/opt/anaconda3/envs/qsp/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /Users/jdocter/opt/anaconda3/envs/qsp/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:1209 strided_slice
        op = gen_array_ops.stri